# 引言：为什么使用 Pipeline？

在实际的机器学习项目中，我们通常会经历如下流程：

1. 数据清洗与预处理（如标准化、缺失值处理、编码）
2. 模型训练与验证
3. 模型评估与调参
4. 模型部署

如果每个步骤分开手动实现，会带来如下问题：
- 📌 **代码冗余**：不同阶段重复写标准化、编码等步骤
- 📌 **流程不一致**：训练和测试数据的处理容易不一致
- 📌 **易出错**：在交叉验证中可能“提前泄露”测试集信息（Data Leakage）
- 📌 **不便复现**：训练流程无法端到端复现，影响部署与团队协作

---

## ✅ 什么是 Pipeline？

`Pipeline` 是 Scikit-learn 提供的一个实用类，它将多个数据处理步骤和模型封装为一个整体流程：

```python
from sklearn.pipeline import Pipeline

Pipeline([
    ("step1", transformer1),   # 预处理步骤，例如 StandardScaler()
    ("step2", transformer2),   # 可选的中间处理
    ("model", estimator)       # 最后一步是模型，例如 LogisticRegression()
])

## ✅ Pipeline 的核心优势

| 优势                         | 说明                                                                 |
|----------------------------|----------------------------------------------------------------------|
| **流程统一**                 | 保证训练集与测试集使用完全相同的预处理步骤                             |
| **结构清晰可维护**           | 多步骤逻辑被模块化、可复用、可读性强                                   |
| **防止信息泄露（leakage）** | 所有步骤都在 `.fit()` 里同时执行，避免处理顺序错误造成的信息泄露         |
| **支持调参**                 | 可通过 `GridSearchCV` 调整任意步骤中的参数，使用命名空间 `step__param` 格式 |
| **便于部署**                 | 整个 pipeline 可用 `joblib.dump()` 保存并部署                           |


# 1. 数据加载与探索

为了演示 Pipeline 的完整流程，我们将使用 `scikit-learn` 自带的乳腺癌二分类数据集（`load_breast_cancer`），它包含数值型特征，适合标准化 + 分类建模。

---

In [2]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# 加载数据
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pd.DataFrame(X, columns=feature_names).head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# 2. 特征预处理：使用 ColumnTransformer

在一个典型的机器学习任务中，数据通常包含不同类型的特征：
- 数值型特征（numerical features）如年龄、收入等
- 类别型特征（categorical features）如性别、职业等

我们需要对它们分别进行不同的预处理操作。

---

## 🧰 工具类：ColumnTransformer

`ColumnTransformer` 是 Scikit-learn 中的工具类，允许我们**对不同列应用不同的预处理器**。

```python
from sklearn.compose import ColumnTransformer


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# 所有列都进行标准化
numerical_cols = list(range(X.shape[1]))

preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), numerical_cols)
])


## ℹ️ 参数解释

| 参数名              | 说明                                                                 |
|---------------------|----------------------------------------------------------------------|
| `transformers`      | 列表，包含多个三元组 `(名称, 变换器, 作用列)`                        |
| `remainder='drop'`  | 默认值，未列出的列会被丢弃；可设为 `'passthrough'` 保留原始值        |
| `handle_unknown`    | 对于 OneHotEncoder 等分类器常设为 `'ignore'` 避免测试集出现未知值报错 |


# 3️⃣ 构建 Pipeline：封装预处理与模型

完成了预处理器（如标准化）之后，我们希望将其与模型组合在一起构建一个完整的工作流。  
这就是 `Pipeline` 的核心作用 —— **将多个步骤按顺序封装为一个整体对象**。

---

## 🔧 Pipeline 是什么？

`Pipeline` 是 `sklearn.pipeline` 提供的类，用于将多个步骤组合起来依次执行。

```python
from sklearn.pipeline import Pipeline
```

## ✅ Pipeline 的结构

```python
pipeline = Pipeline([
    ('preprocessor', preprocessor),           # Transformer：比如 ColumnTransformer
    ('classifier', LogisticRegression())      # Estimator：分类器或回归器
])
```

注意：

所有步骤必须是 (名称, 对象) 的二元组；

除了最后一个步骤必须是模型（实现 fit() 和 predict()），前面的必须是 transformer（实现 fit() 和 transform()）；

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])


# 4️⃣ 模型训练与评估

构建好 `Pipeline` 之后，我们可以使用它来训练模型并在测试集上进行评估。

由于 `Pipeline` 本质上是一个模型对象，它也拥有常用接口如 `.fit()`、`.predict()`、`.score()` 等，我们可以像使用任何模型那样使用它。

---

## 🧪 训练模型

```python
pipeline.fit(X_train, y_train)

```

该命令会自动执行以下步骤：

对 X_train 应用所有预处理器（如标准化、编码等）

用处理后的特征训练模型（如逻辑回归）


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9736842105263158
Confusion Matrix:
 [[41  2]
 [ 1 70]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



# 5️⃣ Pipeline 总结 + 手动实现对比

---

## ✅ Pipeline 的优势总结

通过前几节内容我们可以看到，Pipeline 的使用带来了诸多优势：

| 优势                       | 说明                                                                 |
|--------------------------|----------------------------------------------------------------------|
| **统一训练流程**           | `.fit()` 一次性完成预处理 + 训练                                     |
| **防止数据泄露**           | 自动确保 test 数据未参与训练中的任何统计信息                         |
| **结构清晰可维护**         | 所有处理步骤集中封装，代码清晰、模块化                              |
| **便于调参与部署**         | 支持 `GridSearchCV`；可序列化保存整个流程                          |
| **节省代码 + 易于扩展**     | 少写很多变量命名和重复处理逻辑                                      |

---


In [10]:
## ❌ 如果不用 Pipeline，我们该如何手动实现？

##下面是一个等价实现，用 Pandas + Scikit-learn 手动完成与 Pipeline 相同的流程。

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

# 加载数据
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# 划分训练测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🧼 手动标准化：注意必须用训练集 fit，测试集 transform
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🧠 训练模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

# 📈 预测与评估
y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9736842105263158
Confusion Matrix:
 [[41  2]
 [ 1 70]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

